In [2]:
# load necessary modules

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import category_encoders as ce
import mlflow

from sklearn import metrics
from sklearn import datasets
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# load data
df = pd.read_csv('../data/penguins.csv')

# data preprocessing
df=df.drop_duplicates()
df.dropna(axis=0,inplace=True)

# one hot encoding - nominal data
encoder=ce.OneHotEncoder(cols=['species','island'],handle_unknown='return_nan', return_df=True,use_cat_names=True)
df_enc=encoder.fit_transform(df)

# add a new column with labels
df_enc.loc[df_enc.sex == 'male', 'label'] = int(1)
df_enc.loc[df_enc.sex == 'female', 'label'] = int(0)
df_enc['label'].astype('float')

# check number of males/females to inspect if dataset is imbalanced
df_enc['label'].value_counts()

# get labels
labels = df_enc[['label']]
y = labels.to_numpy().reshape(-1,)

# drop useless columns
df_enc.drop(columns=['rowid','sex'],axis=1,inplace=True)

# get features
X = df_enc.iloc[:,:11].values

# split dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=True, stratify=y, random_state=42)

mlflow.sklearn.autolog()
mlflow.set_experiment('penguins_log_reg_v1')

# standartization, fitting and estimation of the errors
scaler = StandardScaler()
log_reg = LogisticRegression(C=0.01,
                                    fit_intercept=True, intercept_scaling=1,
                                    max_iter=100,
                                    multi_class='ovr', 
                                    penalty='l2', 
                                    solver='lbfgs', tol=0.0001)
pipe_def_model = Pipeline([('scaler', scaler), ('log_reg', log_reg)])
pipe_def_model.fit(X_train, y_train)
err_train_def_model = pipe_def_model.score(X_train, y_train)
err_val_def_model = pipe_def_model.score(X_val, y_val)
y_pred_def_model = pipe_def_model.predict(X)
accuracy_def_model = pipe_def_model.score(X, y)

# printing
print(f"Accuracy of classification: {round(100*accuracy_def_model, 2)}%")
print(f"Training error: {err_train_def_model}")
print(f"Validation error: {err_val_def_model}")

2022/08/24 15:02:10 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '656585301a8b42149c001e084d37bc62', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2022/08/24 15:02:12 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/mikhailkuklin/miniforge3/envs/mlflow_tacking-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."


Accuracy of classification: 90.69%
Training error: 0.9135338345864662
Validation error: 0.8805970149253731
